In [1]:
from datasets import load_dataset
import Preprocessor as p
import numpy as np
import nltk
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt
from transformers import pipeline
from datasets import load_dataset
from tqdm import tqdm
import re
import torch.nn.functional as  F
import csv
from transformers import AutoModelForCausalLM, AutoTokenizer
from helpMe import get_ppl, convert_to_float, getList,train_rf, train_loop, Average, map_boolean, generate_embeddings, write_embeddings_to_csv
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from customDataset import CustomDataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from modelNN import SimpleNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, KFold


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/oliver/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [28]:
# Load the copenlu/answerable_tydiqa dataset
dataset = load_dataset("copenlu/answerable_tydiqa")

train_df = dataset['train'].to_pandas()
train_df = train_df[train_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
train_df = p.DataFramePreprocessor(train_df).df

val_df = dataset['validation'].to_pandas()
val_df = val_df[val_df['language'].isin(['indonesian', 'arabic', 'bengali'])]
val_df = p.DataFramePreprocessor(val_df).df


In [29]:
data = train_df
data['document_length'] = data['document_plaintext'].str.len()
data['is_answerable'] = data['annotations'].apply(lambda x: x.get('answer_start', [-1])[0] != -1)

val_df['is_answerable'] = data['annotations'].apply(lambda x: x.get('answer_start', [-1])[0] != -1)


In [30]:
data['is_answerable'] = data['is_answerable'].apply(map_boolean)

In [31]:
bengali = data[data['language'] == 'bengali']
arabic = data[data['language'] == 'arabic']
indonesian = data[data['language'] == 'indonesian']


bengali_val = val_df[val_df['language'] == 'bengali']
arabic_val = val_df[val_df['language'] == 'arabic']
indonesian_val = val_df[val_df['language'] == 'indonesian']


In [32]:
arab_embeddings = pd.read_csv('embeddings_arabic.csv',header=None)
arab_embeddings = arab_embeddings[1]

bengali_embeddings = pd.read_csv('embeddings_bengali.csv',header=None)
bengali_embeddings = bengali_embeddings[1]

indo_embeddings = pd.read_csv('embeddings.csv',header=None)
indo_embeddings = indo_embeddings[1]

In [33]:
def convert_to_numpy_array(s):
    numeric_values = re.findall(r'-?\d+\.\d+', s)
    numpy_array = np.array(numeric_values, dtype=float)
    return numpy_array


bengali_array = bengali_embeddings.apply(convert_to_numpy_array)
arabic_array =  arab_embeddings.apply(convert_to_numpy_array)
indonesian_array =  indo_embeddings.apply(convert_to_numpy_array)



In [34]:
benagali_y = np.array(bengali['is_answerable'])
arabic_y = np.array(arabic['is_answerable'])
indonesian_y = np.array(indonesian['is_answerable'])

In [9]:
import numpy as np

def measure_empirical_randomness(arr):
    
    length = len(arr)
    random_sequence = np.random.randint(2, size=length)
    matching_elements = np.sum(arr == random_sequence)
    empirical_randomness = matching_elements / length
    
    return empirical_randomness


In [10]:
beng_random = measure_empirical_randomness(benagali_y)

In [11]:
print(type(bengali_array[0]))
print(type(benagali_y[0]))

<class 'numpy.ndarray'>
<class 'numpy.int64'>


In [12]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [13]:


class CustomDataset(Dataset):
    def __init__(self, input_data, targets):
        self.input_data = torch.FloatTensor(input_data)
        self.targets = torch.FloatTensor(targets)
        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        input_sample = self.input_data[idx]
        target = self.targets[idx]
        return input_sample, target
    
    def split_dataset(self, split_ratio=0.8, random_seed=None):

        # Calculate the sizes of training and validation sets based on split_ratio
        train_size = int(split_ratio * len(self))
        val_size = len(self) - train_size
        
        # Use random_split to split the dataset into training and validation subsets
        train_dataset, val_dataset = random_split(self, [train_size, val_size], generator=torch.Generator().manual_seed(random_seed))
        
        return train_dataset, val_dataset


In [17]:
batch_size = 32  

bengali_data = CustomDataset(bengali_array, benagali_y)
bengali_train, bengali_test = bengali_data.split_dataset(split_ratio=0.8, random_seed=42)
train_loader_bengali = DataLoader(bengali_train, batch_size=batch_size, shuffle=True)
val_loader_bengali = DataLoader(bengali_test, batch_size=batch_size, shuffle=False)


arab_data = CustomDataset(arabic_array, arabic_y)
arab_train, arab_test = arab_data.split_dataset(split_ratio=0.8, random_seed=42)
train_loader_arab = DataLoader(arab_train, batch_size=batch_size, shuffle=True)
val_loader_arab = DataLoader(arab_test, batch_size=batch_size, shuffle=False)

indonesian_data = CustomDataset(indonesian_array, indonesian_y)
indonesian_train, indonesian_test = indonesian_data.split_dataset(split_ratio=0.8, random_seed=42)
train_loader_indonesian = DataLoader(indonesian_train, batch_size=batch_size, shuffle=True)
val_loader_indonesian = DataLoader(indonesian_test, batch_size=batch_size, shuffle=False)


/tmp/ipykernel_14386/2824331789.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  self.input_data = torch.FloatTensor(input_data)


In [18]:
model = SimpleNN(768,128,256,128, 1).to(device)


In [19]:
beng_loss = train_loop(model, "bengali_model", train_loader_bengali)
#arab_loss = train_loop(model, "arab_model")
#indo_loss = train_loop(model, "indo_model", train_loader_indonesian)

Epoch [1/400], Loss: 0.6373
Epoch [2/400], Loss: 0.5681
Epoch [3/400], Loss: 0.5574
Epoch [4/400], Loss: 0.5470
Epoch [5/400], Loss: 0.5400
Epoch [6/400], Loss: 0.5297
Epoch [7/400], Loss: 0.5202
Epoch [8/400], Loss: 0.5241
Epoch [9/400], Loss: 0.5093
Epoch [10/400], Loss: 0.5105
Epoch [11/400], Loss: 0.5039
Epoch [12/400], Loss: 0.4989
Epoch [13/400], Loss: 0.4922
Epoch [14/400], Loss: 0.4864
Epoch [15/400], Loss: 0.4779
Epoch [16/400], Loss: 0.4793
Epoch [17/400], Loss: 0.4807
Epoch [18/400], Loss: 0.4670
Epoch [19/400], Loss: 0.4753
Epoch [20/400], Loss: 0.4503
Epoch [21/400], Loss: 0.4500
Epoch [22/400], Loss: 0.4551
Epoch [23/400], Loss: 0.4320
Epoch [24/400], Loss: 0.4357
Epoch [25/400], Loss: 0.4469
Epoch [26/400], Loss: 0.4478
Epoch [27/400], Loss: 0.4256
Epoch [28/400], Loss: 0.4276
Epoch [29/400], Loss: 0.4145
Epoch [30/400], Loss: 0.4305
Epoch [31/400], Loss: 0.4272
Epoch [32/400], Loss: 0.4003
Epoch [33/400], Loss: 0.4113
Epoch [34/400], Loss: 0.4272
Epoch [35/400], Loss: 0

Epoch [278/400], Loss: 0.2603
Epoch [279/400], Loss: 0.2411
Epoch [280/400], Loss: 0.2412
Epoch [281/400], Loss: 0.2035
Epoch [282/400], Loss: 0.2701
Epoch [283/400], Loss: 0.2893
Epoch [284/400], Loss: 0.2212
Epoch [285/400], Loss: 0.2194
Epoch [286/400], Loss: 0.2497
Epoch [287/400], Loss: 0.2274
Epoch [288/400], Loss: 0.2781
Epoch [289/400], Loss: 0.2424
Epoch [290/400], Loss: 0.2201
Epoch [291/400], Loss: 0.2573
Epoch [292/400], Loss: 0.2068
Epoch [293/400], Loss: 0.2412
Epoch [294/400], Loss: 0.2446
Epoch [295/400], Loss: 0.2160
Epoch [296/400], Loss: 0.2089
Epoch [297/400], Loss: 0.2054
Epoch [298/400], Loss: 0.2264
Epoch [299/400], Loss: 0.2471
Epoch [300/400], Loss: 0.2289
Epoch [301/400], Loss: 0.2214
Epoch [302/400], Loss: 0.2126
Epoch [303/400], Loss: 0.2372
Epoch [304/400], Loss: 0.2308
Epoch [305/400], Loss: 0.2467
Epoch [306/400], Loss: 0.2105
Epoch [307/400], Loss: 0.2146
Epoch [308/400], Loss: 0.2068
Epoch [309/400], Loss: 0.2117
Epoch [310/400], Loss: 0.2040
Epoch [311

In [19]:
import torch
from sklearn.metrics import precision_score

def evaluate_precision_with_pretrained(model, pretrained_weights_path, val_loader, device):
    # Load the pre-trained weights
    model.load_state_dict(torch.load(pretrained_weights_path))
    model.to(device)
    model.eval()  # Set the model to evaluation mode

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device, dtype=torch.float32), targets.to(device, dtype=torch.float32)
            outputs = model(inputs).squeeze()  # Squeeze the output to remove singleton dimensions

            preds = (outputs > 0.5).float()  # Convert probabilities to binary predictions
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    precision = precision_score(all_targets, all_preds)
    return precision

In [20]:
#engali_precision = evaluate_precision_with_pretrained(model,'bengali_model',val_loader_bengali, device)
print(f'Precision on validation of Bengali: {bengali_precision:.4f}')

#ndonesian_precision = evaluate_precision_with_pretrained(model,'indo_model',val_loader_indonesian, device)
print(f'Precision on validation of Indonesian: {indonesian_precision:.4f}')


NameError: name 'bengali_precision' is not defined

In [ ]:
def train_rf(xs, ys, lang):
    flattened_data = np.array([sub_array.flatten() for sub_array in xs])
    X_train, X_test, y_train, y_test = train_test_split(flattened_data, ys, test_size=0.1, random_state=42)
    
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    rf_classifier = RandomForestClassifier(random_state=42)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, 
                               scoring='accuracy', cv=kf, n_jobs=-1)

    grid_search.fit(X_train, y_train)
    print('Best Accuracy: ', lang, ' ', grid_search.best_score_ * 100)
    #print(f"Best Accuracy {lang}: {:.2f}%".format(grid_search.best_score_ * 100))




In [ ]:

#train_rf(bengali_array, benagali_y, 'bengali')
#train_rf(indonesian_array, indonesian_y, 'indonesian')

In [24]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder






In [16]:
from xgboost import XGBClassifier

def train_xgb_model(xs, ys):
    flattened_data = np.array([sub_array.flatten() for sub_array in xs])
    X_train, X_test, y_train, y_test = train_test_split(flattened_data, ys, test_size=0.1, random_state=42)
    
    # Define hyperparameters for the XGBoost classifier
    params = {
        'objective': 'multi:softmax',  # Multiclass classification problem
        'num_class': len(set(ys)),    # Number of classes
        'n_estimators': 100,           # Number of boosting rounds
        'learning_rate': 0.1,         # Learning rate
        'max_depth': 6                 # Maximum depth of each tree
    }

    # Create and fit the XGBoost classifier
    classifier = XGBClassifier(**params)
    classifier.fit(X_train, y_train)

    return classifier

# Example usage:
# trained_classifier = train_xgb_model(xs, ys)


In [42]:
#model_beng = train_xgb_model(bengali_array, benagali_y)#
model_indo = train_xgb_model(indonesian_array, indonesian_y)
model_arabic = train_xgb_model(arabic_array, arabic_y)

In [54]:
flattened_data = np.array([sub_array.flatten() for sub_array in bengali_array]) 
X_train, X_test, y_train, y_test = train_test_split(flattened_data, benagali_y, test_size=0.1, random_state=42)

In [53]:
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split

def evaluate_xgb_model(xs, ys, lang, model):
    flattened_data = np.array([sub_array.flatten() for sub_array in xs])
    X_train, X_test, y_train, y_test = train_test_split(flattened_data, ys, test_size=0.1, random_state=42)
    
    # Assuming you have trained and made predictions using your XGBoost model
    predictions = model.predict(X_test)

    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')  # Precision score
    recall = recall_score(y_test, predictions, average='weighted')  # Recall score
    f1 = f1_score(y_test, predictions, average='weighted')  # F1 score

    print(f"Test Accuracy ({lang}): {accuracy * 100:.2f}%")
    print(f"Precision ({lang}): {precision:.2f}")
    print(f"Recall ({lang}): {recall:.2f}")
    print(f"F1 Score ({lang}): {f1:.2f}")

    return accuracy, precision, f1, lang


In [55]:
acc = evaluate_xgb_model(X_test, y_test, 'bengali', model_beng)

Test Accuracy (bengali): 83.33%
Precision (bengali): 0.84
Recall (bengali): 0.83
F1 Score (bengali): 0.83


In [56]:
flattened_data = np.array([sub_array.flatten() for sub_array in indonesian_array]) 
X_train, X_test, y_train, y_test = train_test_split(flattened_data, indonesian_y, test_size=0.1, random_state=42)

In [57]:
acc = evaluate_xgb_model(X_test, y_test, 'indonesian', model_indo)

Test Accuracy (indonesian): 69.30%
Precision (indonesian): 0.69
Recall (indonesian): 0.69
F1 Score (indonesian): 0.69


In [58]:
flattened_data = np.array([sub_array.flatten() for sub_array in arabic_array]) 
X_train, X_test, y_train, y_test = train_test_split(flattened_data, arabic_y, test_size=0.1, random_state=42)

In [59]:
acc = evaluate_xgb_model(X_test, y_test, 'arabic', model_arabic)

Test Accuracy (arabic): 80.41%
Precision (arabic): 0.81
Recall (arabic): 0.80
F1 Score (arabic): 0.80
